In [ ]:
 #this query identifies, individuals with a code for being an ex-smoker in the care home cohort, it only identifies the most recent code to admission to the care home. i then did a union all with the current smoker equivalent
CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_smoking_status`as (

#this retrieves all ctv3codes for the care home cohort
with b as (
  SELECT person_id, first_episodestartdate
  from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`),

person as (
    SELECT 
  person_id,
  ctv3code,
  ctv3text,
  dateevent
  FROM`yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  WHERE person_id in(SELECT person_id FROM `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),


ex_all as (
  SELECT 
    person.person_id,
    person.ctv3code,
    person.ctv3text,
    person.dateevent as exsmoke_date,
    cast(b.first_episodestartdate as datetime) ch_entry_date
    from person 
    LEFT JOIN  b
    on person.person_id = b.person_id
  WHERE person.ctv3code in(SELECT CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where Deficit= 'Smoker (ex)')),
  
  ex_All_no as (
  Select 
  *,
  row_number() over (partition by person_id order by exsmoke_date desc) as exsmoke_seq
  from ex_all
  Where ex_all.exsmoke_date < ex_all.ch_entry_date),

  ex_filtered as (
    SELECT 
    *
    from ex_all_no
  where exsmoke_seq = 1),

  current_all as (  
   SELECT 
    person.person_id,
    person.ctv3code,
    person.ctv3text,
    person.dateevent as current_smoke_date,
    cast(b.first_episodestartdate as datetime) ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE ctv3code in(SELECT CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where Deficit= 'Smoker (current)')),
  
  current_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by current_smoke_date desc) as current_smoke_seq
  from current_all
  Where current_smoke_date < ch_entry_date),

  current_filtered as (
    select 
    *
    from current_all_no
  where current_smoke_seq = 1),

  non_smoke_all as (
    SELECT 
    person.person_id,
    person.ctv3code,
    person.ctv3text,
    person.dateevent as non_smoke_date,
    cast(b.first_episodestartdate as datetime) ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE ctv3code in(SELECT CTV3_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_non_smoker_snomed_ctv3`)),

  non_smoke_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by non_smoke_date desc) as non_smoke_seq
  from non_smoke_all
  Where non_smoke_date < ch_entry_date),

  non_smoke_filtered as (
  SELECT
  *
  from non_smoke_all_no
  where non_smoke_seq = 1),

  merged_smoking as (
  
  SELECT 
  * from ex_filtered

  UNION ALL 

  select * from current_filtered

  UNION ALL 

  select * from non_smoke_filtered),

merged_smoking_filtered as (
select
*,
row_number() over (partition by person_id order by exsmoke_date desc) as smoke_seq
from merged_smoking)


  select
   *,
   CASE when ctv3code in(SELECT CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where Deficit= 'Smoker (ex)') then 'ex-smoker'
   when ctv3code in(SELECT CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where Deficit= 'Smoker (current)') then 'current smoker'
   when ctv3code in(SELECT CTV3_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_non_smoker_snomed_ctv3`) then 'non-smoker'
   else "missing"
   END as smoking_status
  from merged_smoking_filtered
  where smoke_seq = 1
  order by person_id)



